# How to use the RNN Disaggregator with NILMTK

This is an example on how to train and use the Recurrent Network (RNN) disaggregator on the [REDD](http://redd.csail.mit.edu/) dataset using [NILMTK](https://github.com/nilmtk/NILMTK/).

This network was described in the [Neural NILM](https://arxiv.org/pdf/1507.06594.pdf) paper.

First of all, we need to train the RNNDisaggregator using the train data. For this example, both train and test data are consumption data of the microwave of the first REDD building.

In [4]:
import warnings; warnings.filterwarnings('ignore')

from nilmtk import DataSet
train = DataSet('../../SeniorDataset/h5_files/96Hour.h5')
train.set_window(start='2022-01-00T00', end='2022-01-05T00')
train_elec = train.buildings[1].elec

OSError: No such file as /h5_files/96Hour.h5

Next, we need to define the disaggregator model.

In [ ]:
from rnndisaggregator import RNNDisaggregator
rnn = RNNDisaggregator()

Then train the model. We need to input the train data as well as their sample period. Also, we need to pass the desired number of training epochs. Finally, save the model for later use.

In [ ]:
train_mains = train_elec.mains() # The aggregated meter that provides the input
print(train_mains)
train_meter = train_elec.submeters()['kettle'] # The microwave meter that is used as a training target

rnn.train(train_mains, train_meter, epochs=1, sample_period=1)
rnn.export_model("model-ukDaleTEST.h5")

Now that the model is trained, we can use it to disaggregate energy data. Let's test it on the rest of the data from building 1.

First we use the model to predict the microwave consumption. The results are saved automatically in a .h5 datastore.

In [ ]:
test = DataSet('ukdale.h5')
test.set_window(start="1-3-2014", end="1-4-2014")
test_elec = test.buildings[1].elec
test_mains = test_elec.mains()

disag_filename = 'disag-outUKDALE_TEST.h5' # The filename of the resulting datastore
from nilmtk.datastore import HDFDataStore
output = HDFDataStore(disag_filename, 'w')

# test_mains: The aggregated signal meter
# output: The output datastore
# train_meter: This is used in order to copy the metadata of the train meter into the datastore
rnn.disaggregate(test_mains, output, train_meter, sample_period=1)
output.close()

Let's plot the results and compare them to the ground truth signal.

**Note:** Calling plot this way, downsamples the signal to reduce computing time. To plot the entire signal call
```
predicted.power_series_all_data().plot()
ground_truth.power_series_all_data().plot()
```

In [ ]:
result = DataSet(disag_filename)
res_elec = result.buildings[1].elec
predicted = res_elec['kettle']
ground_truth = test_elec['kettle']

import matplotlib.pyplot as plt
predicted.plot(plot_kwargs={'color':'red', 'label':'Predicted'})
ground_truth.plot(plot_kwargs={'color':'green', 'label':'Ground Truth'})
plt.show()

Finally let's see the metric results.

In [ ]:
import metrics
rpaf = metrics.recall_precision_accuracy_f1(predicted, ground_truth)
print("============ Recall: {}".format(rpaf[0]))
print("============ Precision: {}".format(rpaf[1]))
print("============ Accuracy: {}".format(rpaf[2]))
print("============ F1 Score: {}".format(rpaf[3]))

print("============ Relative error in total energy: {}".format(metrics.relative_error_total_energy(predicted, ground_truth)))
print("============ Mean absolute error(in Watts): {}".format(metrics.mean_absolute_error(predicted, ground_truth)))